In [2]:
import numpy as np
from qiskit.quantum_info import random_clifford, Pauli

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

def tensor_prod(*tensors):
    if len(tensors) == 2:
        return np.kron(tensors[0], tensors[1])
    else:
        return np.kron(tensors[0], tensor_prod(*tensors[1:]))
    
def hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

def trace_one(matrix):
    return np.isclose(np.trace(matrix), 1)

def positive_semi_definite(matrix, tol=1e-8):
    return np.all(np.linalg.eigvals(matrix) + tol >= 0)

def is_legal(matrix):
    return hermitian(matrix) and trace_one(matrix) and positive_semi_definite(matrix)

def int_to_bin_list(n, length):
    bin_list = np.zeros(length)
    bin_list[n] = 1
    return bin_list

def single_sample(prob_list):
    assert np.isclose(sum(prob_list), 1), "probability does not sum up to 1"
    rd = np.random.random()
    inf, sup = 0, 0
    for i, e in enumerate(prob_list):
        sup += e
        if inf <= rd <= sup:
            return i
        else:
            inf = sup
    raise ValueError("random value does not meet any interval")

class QuantumState():
    def __init__(self, num_qubits:int, num_shots:int, batch_size:int, pauli_observables:list, veri:bool):
        self._num_qubits = num_qubits
        self._observables = pauli_observables
        self._batch_size = batch_size
        self._num_shots = num_shots
        self._veri = veri
        self._dm = None
        self._entangled = None
        
    @property
    def dm(self):
        return self._dm
    
    @dm.setter
    def dm(self, new_dm):
        if not (self._veri or is_legal(new_dm)):
            raise ValueError("density matrix is not physical")
        else:
            self._dm = new_dm
    
    def set_dm(self):
        raise NotImplementedError("without information to construct density matrix")
    
    def random_evolve(self):
        self._U = random_clifford(self._num_qubits).to_matrix()
        self._dm = self._U @ self.dm @ np.conj(self._U).T
    
    def single_shot_measure(self):
        prob_list = [self._dm[i, i] for i in range(2 ** self._num_qubits)]
        single_shot_state = int_to_bin_list(single_sample(prob_list), 2 ** self._num_qubits)
        del self._dm
        self._state = single_shot_state
    
    def reconstruct_dm(self):
        dim = 2 ** self._num_qubits
        return (dim + 1) * (np.conj(self._U).T @ np.outer(self._state, self._state) @ self._U) - np.eye(dim)

    # def classical_shadow(self):
    #     shadows = {obs: [] for obs in self._observables}
    #     temp_shadows = {obs: [] for obs in self._observables}
    #     dm_copy = self._dm
    #     for _ in range(self._num_shots // self._batch_size):
    #         for _ in range(self._batch_size):
    #             self._dm = dm_copy
    #             self.random_evolve()
    #             self.single_shot_measure()
    #             rdm = self.reconstruct_dm()
    #             for k, v in temp_shadows.items():
    #                 v.append(np.trace(Pauli(k).to_matrix() @ rdm))
    #         for k, v in shadows.items():
    #             v.append(np.mean(temp_shadows[k]))
    #         temp_shadows = {obs: [] for obs in self._observables}
    #     del temp_shadows
    #     return {k: np.median(v) for k, v in shadows.items()}
    
    def classical_shadow(self):
        shadows = {obs: [] for obs in self._observables}
        dm_copy = self._dm
        for _ in range(self._num_shots // self._batch_size):
            snapshots = []
            for _ in range(self._batch_size):
                self._dm = dm_copy
                self.random_evolve()
                self.single_shot_measure()
                snapshots.append(self.reconstruct_dm())
            mean = np.mean(np.stack(snapshots), axis=0)
            for k, v in shadows.items():
                v.append(np.trace(Pauli(k).to_matrix() @ mean))
        return {k: np.median(v) for k, v in shadows.items()}

In [3]:
def partial_transpose(matrix):
    if matrix.shape != (4, 4):
        raise ValueError("Input matrix must be 4x4.")
    result = np.zeros((4, 4), dtype=matrix.dtype)
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    result[2*i + k, 2*j + l] = matrix[2*i + k, 2*j + l]
                    result[2*i + k, 2*j + l] = matrix[2*i + l, 2*j + k]
    return result

state_01 = np.array([[0], 
                     [1],
                     [0],
                     [0]])
state_10 = np.array([[0], 
                     [0],
                     [1],
                     [0]])

In [4]:
class Wernerlikestate(QuantumState):
    def __init__(self, p, theta, phi, num_qubits:int, num_shots:int, batch_size:int, pauli_observables:list, veri:bool):
        super().__init__(num_qubits, num_shots, batch_size, pauli_observables, veri)
        assert num_qubits == 2, "Werner-like states contain only 2 qubits"
        self._p = p
        self._theta = theta
        self._phi = phi
        assert 0 <= p <= 1, "Werner-like state parameter F must lie between 0 and 1"

    def set_dm(self):
        psi = np.cos(self._theta) * state_01 + np.exp(1j * self._phi) * np.sin(self._theta) * state_10
        # if self._p == 1:
        #     if is_legal(tensor_prod(psi, np.conj(psi).T)):
        #         self._dm = tensor_prod(psi, np.conj(psi).T)
        #         return self._dm
        #     else:
        #         raise ValueError("wtf")
        new_dm = self._p * tensor_prod(psi, np.conj(psi).T) + (1 - self._p) * np.eye(4) / 4
        if is_legal(new_dm):
            self._dm = new_dm
            return self._dm
        else:
            print(f"error occur: p={self._p}, theta={self._theta}, phi={self._phi}")
            raise NotImplementedError("density matrix setting wrongly implemented")
    
    @property
    def entangled(self):
        pt_dm = partial_transpose(self._dm)
        return not positive_semi_definite(pt_dm)
               

In [5]:
test_state = Wernerlikestate(.0, .0, .0, 2, 10000, 25, ['XX', 'YY', 'ZZ'], False)
test_state.set_dm()
test_state.entangled

False

In [8]:
# Initial parameters
np.random.seed(42)
ps = np.linspace(0, 1, 1000)
thetas = np.linspace(0, np.pi, 10)
phis = np.linspace(0, 2 * np.pi, 10)
grid1, grid2, grid3 = np.meshgrid(ps, thetas, phis)
parameters = np.vstack([grid1.ravel(), grid2.ravel(), grid3.ravel()]).T
observables = ['XY', 'ZZ', 'ZI']

# Generate dataset
dataset = []
for parameter in parameters:
    temp_state = Wernerlikestate(*parameter, 2, None, None, None, False) # use theoretical value here
    temp_dm = temp_state.set_dm()
    features = [np.trace(temp_dm @ Pauli(obs).to_matrix()).real for obs in observables]
    result = temp_state.entangled
    dataset.append(np.append(features, result))
del temp_state
del temp_dm
dataset = pd.DataFrame(dataset, columns = observables + ['result'])
X = dataset.drop('result', axis=1)
y = dataset['result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     11001
         1.0       1.00      1.00      1.00      8999

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000



In [7]:
dataset.to_csv('werner_like_data.csv', index=False)

In [25]:
shadow_dataset = pd.read_csv("output\werner_like_shadow_data.csv")
shadow_X = shadow_dataset.drop('result', axis=1)
def replace_0j(x):
    return np.nan if x == 0j else complex(x).real
shadow_X = shadow_X.map(replace_0j)
shadow_y = shadow_dataset['result'].values
shadow_y_pred = model.predict(shadow_X)
accuracy = accuracy_score(shadow_y, shadow_y_pred)
report = classification_report(shadow_y, shadow_y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Accuracy: 0.87
Classification Report:
              precision    recall  f1-score   support

       False       1.00      0.83      0.91       930
        True       0.67      1.00      0.80       320

    accuracy                           0.87      1250
   macro avg       0.83      0.91      0.85      1250
weighted avg       0.92      0.87      0.88      1250

